In [21]:
# Set some notebook defaults
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20;

# Display information about the environment
VERSION, Base.Threads.nthreads(), Base.Sys.CPU_THREADS

(v"1.5.3", 8, 8)

In [22]:
using Random, Distributions, BenchmarkTools, Tullio, LoopVectorization, DataFrames, StaticKernels, SantasLittleHelpers

In [23]:
N = 10^8
d = Normal()
df = DataFrame(:data => rand(d, N))

,data
,Float64
1,-1.71653
2,-0.854815
3,-1.32994
4,0.146672
5,-0.0621817
6,-0.776147
7,-0.300822
8,-1.21474
9,-0.57942


In [24]:
# Calculate sma using StaticKernels and SantasLittleHelpers
# Contains code originally from 

k = makekernel(mean,-9:0)
df[!, "sma_sk"] = applyrolling(k, df.data)
df

,data,sma_sk
,Float64,Float64
1,-1.71653,-1.71653
2,-0.854815,-1.28567
3,-1.32994,-1.30043
4,0.146672,-0.938653
5,-0.0621817,-0.763359
6,-0.776147,-0.76549
7,-0.300822,-0.699109
8,-1.21474,-0.763563
9,-0.57942,-0.743103


In [25]:
# Contains code originally from Michael Abbott

tmp = Array{Union{Missing, Float64}}(missing, length(df.data));

function f_tullio(data, sma)
    out_view = @view sma[10:end]
    @tullio out_view[i] = 0.1*data[i+o]  o in 0:9  # verbose=true;
    sma
end

df[!, "sma_tullio"] = f_tullio(df.data, tmp)
df

,data,sma_sk,sma_tullio
,Float64,Float64,Float64?
1,-1.71653,-1.71653,missing
2,-0.854815,-1.28567,missing
3,-1.32994,-1.30043,missing
4,0.146672,-0.938653,missing
5,-0.0621817,-0.763359,missing
6,-0.776147,-0.76549,missing
7,-0.300822,-0.699109,missing
8,-1.21474,-0.763563,missing
9,-0.57942,-0.743103,missing


In [26]:
# Contains code originally from Peter Deffebach

out = Vector{Union{Missing, Float64}}(undef, N)

function rollingmean2(x, m, out)
    N = length(x)
    out[1:(m-1)] .= missing
    sm = mean(@view x[1:m])
    out[m] = sm
    @inbounds @simd for i in (m+1):length(x)
        sm += (x[i] - x[i-m]) / m
        out[i] = sm
    end
    return out 
end

rollingmean2 (generic function with 1 method)

In [27]:
df[!, "sma_simd"] = rollingmean2(df.data, 10, out)
df

,data,sma_sk,sma_tullio,sma_simd
,Float64,Float64,Float64?,Float64?
1,-1.71653,-1.71653,missing,missing
2,-0.854815,-1.28567,missing,missing
3,-1.32994,-1.30043,missing,missing
4,0.146672,-0.938653,missing,missing
5,-0.0621817,-0.763359,missing,missing
6,-0.776147,-0.76549,missing,missing
7,-0.300822,-0.699109,missing,missing
8,-1.21474,-0.763563,missing,missing
9,-0.57942,-0.743103,missing,missing


In [28]:
@benchmark applyrolling(k, df.data)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.026 s (0.11% GC)
  median time:      1.055 s (0.12% GC)
  mean time:        1.078 s (3.87% GC)
  maximum time:     1.158 s (10.88% GC)
  --------------
  samples:          5
  evals/sample:     1

In [29]:
@benchmark f_tullio(df.data, tmp)

BenchmarkTools.Trial: 
  memory estimate:  7.58 KiB
  allocs estimate:  114
  --------------
  minimum time:     135.175 ms (0.00% GC)
  median time:      145.747 ms (0.00% GC)
  mean time:        147.128 ms (0.00% GC)
  maximum time:     173.753 ms (0.00% GC)
  --------------
  samples:          34
  evals/sample:     1

In [30]:
@benchmark rollingmean2(df.data, 10, out)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     128.876 ms (0.00% GC)
  median time:      145.206 ms (0.00% GC)
  mean time:        145.148 ms (0.00% GC)
  maximum time:     176.760 ms (0.00% GC)
  --------------
  samples:          35
  evals/sample:     1